In [1]:
import pandas as pd

import numpy as np

import pickle

# Load Datasets

In [2]:
#Reference of the dataset from DeepGO
# ! tar -xvzf  CAFA3_training_data/data12G/data/train.tar.gz

### Load Molecular Function Terms

In [4]:
trainMF_terms=pd.read_pickle('../../../data/DeepGo/train-mf.pkl')

trainMF_terms=trainMF_terms[trainMF_terms['orgs']=='9606']

testMF_terms=pd.read_pickle('../../../data/DeepGo/test-mf.pkl')

testMF_terms=testMF_terms[testMF_terms['orgs']=='9606']

trainMF_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

testMF_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

# load DeepGO embeddings

In [5]:
embedding_size=256

trainProtein_weights = np.zeros((trainMF_terms.shape[0], embedding_size))

testProtein_weights = np.zeros((testMF_terms.shape[0], embedding_size))




In [7]:
trainMF_terms.columns

Index(['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences',
       'orgs', 'embeddings'],
      dtype='object')

In [9]:
for i in range(trainMF_terms.shape[0]):  
    trainProtein_weights[i]=trainMF_terms.iloc[i][7]

for i in range(testMF_terms.shape[0]):  
    testProtein_weights[i]=testMF_terms.iloc[i][7]

In [11]:
X_train=trainProtein_weights
X_train=np.array(X_train)
X_train=np.expand_dims(X_train,axis=-1)

Ytrain=trainMF_terms[['labels']]
Ytrain=Ytrain.labels.apply(pd.Series)
Ytrain=np.array(Ytrain)

X_test=testProtein_weights
X_test=np.array(X_test)
X_test=np.expand_dims(X_test,axis=-1)

Ytest=testMF_terms[['labels']]
Ytest=Ytest.labels.apply(pd.Series)
Ytest=np.array(Ytest)


# +++++++++++++++++++++++++++++++++++++++++++++

### Train and Evaluate the model

In [13]:
import sys
sys.path.append('../../../utils/')

from sklearn.model_selection import StratifiedKFold

from  model import naive_CNN_classifier

import keras

from sklearn.metrics import label_ranking_average_precision_score

from numpy import arange

from sklearn.metrics import f1_score


In [14]:
random_seed=13

In [15]:
num_classes=Ytrain.shape[1]

In [16]:
model=None
model = naive_CNN_classifier(num_classes,embedding_size)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
history=model.fit(X_train, Ytrain, validation_data=(X_test,Ytest) ,epochs=50, batch_size=16,verbose=0)
YtestPredicted_raw=model.predict(X_test)
avePrec =label_ranking_average_precision_score(Ytest, YtestPredicted_raw) 

In [18]:
avePrec

0.46241347556088563

In [19]:
results={'treshold':[],'Average Precision':[],'F1 (micro)':[],'F1 (macro)':[],'Method':[]}

In [20]:
for treshold in arange(0.05,0.55,0.05):  
    YtestPredicted=None
    YtestPredicted=YtestPredicted_raw.copy()
    results['treshold'].append(treshold)
    results['Average Precision'].append(avePrec)
    YtestPredicted[YtestPredicted>=treshold]=1
    YtestPredicted[YtestPredicted<treshold]=0
    results['F1 (micro)'].append  (f1_score(Ytest, YtestPredicted, average='micro'))
    results['F1 (macro)'].append  (f1_score(Ytest, YtestPredicted, average='macro'))
    results['Method'].append ('DeepGO')


/home/ubuntuadmin/anaconda37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [21]:
df_results=pd.DataFrame(results)

In [22]:
df_results.head(50)

,treshold,Average Precision,F1 (micro),F1 (macro),Method
0,0.05,0.462413,0.318908,0.166167,DeepGO
1,0.10,0.462413,0.363112,0.170950,DeepGO
2,0.15,0.462413,0.384953,0.168998,DeepGO
3,0.20,0.462413,0.398945,0.166342,DeepGO
4,0.25,0.462413,0.410853,0.165427,DeepGO
5,0.30,0.462413,0.414301,0.157932,DeepGO
6,0.35,0.462413,0.413908,0.152164,DeepGO
7,0.40,0.462413,0.411453,0.149353,DeepGO
8,0.45,0.462413,0.406714,0.143141,DeepGO
9,0.50,0.462413,0.400695,0.140787,DeepGO


In [24]:
df_results.to_csv('../../../data/_Outputs/Final_DeepGO_MF.csv',index=False)